In [1]:
import pandas as pd
import csv
from collections import defaultdict
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib
import time
import math
from operator import itemgetter
from scipy.spatial import distance
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [2]:
recipe = pd.read_csv('./data/recipe.csv')

In [3]:
recipe = recipe[['recipe_id', 'recipe_name']]
recipe.tail()

,recipe_id,recipe_name
1303,recipe-1304,맥적구이
1304,recipe-1305,채소어묵
1305,recipe-1306,주꾸미보리죽
1306,recipe-1307,오렌지삼겹찜
1307,recipe-1308,소고기볼게살타르


In [5]:
# user_id, rating, recipe_id
ratings = pd.read_csv('./data/ratings.csv')
ratings = ratings[['ratingNo', 'recipe_id', 'user_id', 'rating']]
ratings.tail()

,ratingNo,recipe_id,user_id,rating
39184,39184,recipe-1308,user-88,2
39185,39185,recipe-1308,user-89,2
39186,39186,recipe-1308,user-91,2
39187,39187,recipe-1308,user-94,1
39188,39188,recipe-1308,user-97,3


In [6]:
# user_id
users = pd.read_csv('./data/user.csv')
users = users[['ratingNo','user_id']]
users.tail()

,ratingNo,user_id
95,95,user-96
96,96,user-97
97,97,user-98
98,98,user-99
99,99,user-100


In [7]:
del ratings['ratingNo']
ratings.tail()

,recipe_id,user_id,rating
39184,recipe-1308,user-88,2
39185,recipe-1308,user-89,2
39186,recipe-1308,user-91,2
39187,recipe-1308,user-94,1
39188,recipe-1308,user-97,3


In [8]:
user_num = ratings.user_id.unique().shape[0]
recipe_num = ratings.recipe_id.unique().shape[0]

print("유저의 수는 {}명 이고, \n레시피의 수는 {}개 입니다.".format(user_num, recipe_num))

유저의 수는 100명 이고, 
레시피의 수는 1308개 입니다.


In [10]:
# 사용자 샘플 선출 -> pivot table 생성
UM_matrix_ds = ratings.pivot(index='user_id', columns='recipe_id', values='rating')
UM_matrix_ds.head(5)

recipe_id,recipe-1,recipe-10,recipe-100,recipe-1000,recipe-1001,recipe-1002,recipe-1003,recipe-1004,recipe-1005,recipe-1006,...,recipe-990,recipe-991,recipe-992,recipe-993,recipe-994,recipe-995,recipe-996,recipe-997,recipe-998,recipe-999
user_id,,,,,,,,,,,,,,,,,,,,,
user-1,3.0,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,...,5.0,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN
user-10,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,4.0,4.0,NaN,1.0
user-100,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,2.0,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
user-11,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.0,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,NaN
user-12,1.0,4.0,NaN,2.0,NaN,NaN,2.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,2.0,NaN,5.0,4.0,2.0,NaN


In [11]:
def distance_cosine(a, b) :
       return 1-distance.cosine(a, b)

In [12]:
def distance_correlation(a, b) :
       return 1-distance.correlation(a, b)

In [13]:
def distance_euclidean(a, b) :
       return 1/(1+distance.euclidean(a, b))

In [31]:
def nearest_neighbor_user(user, topN, simFunc) :
    u1  =  UM_matrix_ds.loc[user].dropna()
    ratedIndex = u1.index
    nn = { }
    
    for uid, row in UM_matrix_ds.iterrows():
        interSectionU1 = [ ]
        interSectionU2 = [ ]
        if uid == user: continue
        
        for i in ratedIndex:
            if not math.isnan(row[i]):
                interSectionU1.append(u1[i])
                interSectionU2.append(row[i])
        interSectionLen = len(interSectionU1)
        
        if interSectionLen < 3: continue
        sim = simFunc(interSectionU1, interSectionU2)
        
        if not math.isnan(sim):nn[uid] = sim
                                                
    return sorted(nn.items(), key=itemgetter(1))[:-(topN+1):-1]

In [32]:
for user in range(0, len(UM_matrix_ds)) :
      print('User {0} neighbors : {1}'.format('user-'+str(user+1), nearest_neighbor_user('user-'+str(user+1), 3, distance_cosine)))

User user-1 neighbors : [('user-79', 0.8873525883018792), ('user-75', 0.8826151617831657), ('user-65', 0.8637526728156629)]
User user-2 neighbors : [('user-42', 0.879597155335769), ('user-82', 0.8635368667338414), ('user-53', 0.8558833722757675)]
User user-3 neighbors : [('user-16', 0.8849578627702183), ('user-43', 0.8565254968724393), ('user-19', 0.8541606139626727)]
User user-4 neighbors : [('user-93', 0.8804669431007153), ('user-25', 0.8622221277213945), ('user-49', 0.8611766090660558)]
User user-5 neighbors : [('user-9', 0.8729551763737453), ('user-96', 0.8619480438685505), ('user-55', 0.8598252252087156)]
User user-6 neighbors : [('user-42', 0.8616934300748349), ('user-52', 0.8608053305542304), ('user-60', 0.8538180221532102)]
User user-7 neighbors : [('user-25', 0.85888055455711), ('user-76', 0.8568888888888889), ('user-2', 0.8548594734655425)]
User user-8 neighbors : [('user-66', 0.8714667817894), ('user-46', 0.8662915739369857), ('user-1', 0.862390171890171)]
User user-9 neighb

User user-67 neighbors : [('user-64', 0.8589224129237321), ('user-16', 0.8574532279388077), ('user-57', 0.8475473819321789)]
User user-68 neighbors : [('user-34', 0.8662524225388307), ('user-94', 0.8554535354989778), ('user-81', 0.8543634315367674)]
User user-69 neighbors : [('user-57', 0.8607597439252483), ('user-56', 0.8583869296201797), ('user-1', 0.8519206780307919)]
User user-70 neighbors : [('user-50', 0.8642479194345328), ('user-53', 0.8603009936007606), ('user-76', 0.8553031904486218)]
User user-71 neighbors : [('user-25', 0.8625154147178834), ('user-56', 0.8559314490528386), ('user-82', 0.8511418499493604)]
User user-72 neighbors : [('user-86', 0.859870120093148), ('user-50', 0.8563265178412649), ('user-8', 0.853324932529605)]
User user-73 neighbors : [('user-61', 0.8649110061097874), ('user-21', 0.8594589415600048), ('user-5', 0.859090298663252)]
User user-74 neighbors : [('user-84', 0.8725199116971338), ('user-61', 0.8692822075696746), ('user-46', 0.8618257940290289)]
User u

In [16]:
def predict_rating(userid, nn=100, simFunc=distance_cosine):
    neighbor = nearest_neighbor_user(userid, nn, simFunc)
    neighbor_id = [id for id, sim in neighbor]
    
    neighbor_recipe = UM_matrix_ds.loc[neighbor_id].dropna(1, how='all', thresh=1)
    neighbor_dict = (dict(neighbor))
    ret = []
    
    for recipe_id, row in neighbor_recipe.iteritems():
        jsum, wsum = 0, 0
        for v in row.dropna().iteritems():
            sim = neighbor_dict.get(v[0], 0)
            jsum += sim
            wsum += (v[1]*sim)
        ret.append([recipe_id, wsum/jsum])
        
    return ret

## user - recipe rating 예측 결과

# cosine

In [49]:
result_list = []
for i in range(0, len(users)):
    user_id = users.iloc[i].user_id
#     recipe_id = result_top10.iloc[i].recipe_id
    predict = predict_rating(user_id, 40, distance_cosine)
#     print(predict)
    
    is_users = ratings['user_id'] == user_id
    users_rating = ratings[is_users]
    exist_recipe = users_rating['recipe_id'].tolist()
    
    for recipe in predict:
        result_list.append([user_id, recipe[0], recipe[1]])
    
resultdf = pd.DataFrame(result_list, columns=['user_id','recipe_id','rating'])
resultdf

,user_id,recipe_id,rating
0,user-1,recipe-1,3.494289
1,user-1,recipe-10,4.161142
2,user-1,recipe-100,2.496201
3,user-1,recipe-1000,2.893408
4,user-1,recipe-1001,2.716333
...,...,...,...
130795,user-100,recipe-995,2.362635
130796,user-100,recipe-996,3.063611
130797,user-100,recipe-997,3.153118
130798,user-100,recipe-998,3.297056


In [72]:
predict = pd.merge(left = ratings[['recipe_id','user_id','rating']], right = resultdf, on=['user_id','recipe_id'])
predict

,recipe_id,user_id,rating_x,rating_y
0,recipe-1,user-1,3,3.494289
1,recipe-1,user-10,4,2.771002
2,recipe-1,user-12,1,2.184828
3,recipe-1,user-15,1,2.879270
4,recipe-1,user-19,4,2.878939
...,...,...,...,...
39184,recipe-1308,user-88,2,2.264949
39185,recipe-1308,user-89,2,2.172708
39186,recipe-1308,user-91,2,2.351177
39187,recipe-1308,user-94,1,2.635191


In [83]:
sample_all = pd.merge(left = ratings , right = resultdf, how = "outer", on = ["user_id", 'recipe_id'])
sample_all

,recipe_id,user_id,rating_x,rating_y
0,recipe-1,user-1,3.0,3.494289
1,recipe-1,user-10,4.0,2.771002
2,recipe-1,user-12,1.0,2.184828
3,recipe-1,user-15,1.0,2.879270
4,recipe-1,user-19,4.0,2.878939
...,...,...,...,...
130795,recipe-994,user-100,NaN,2.178000
130796,recipe-995,user-100,NaN,2.362635
130797,recipe-997,user-100,NaN,3.153118
130798,recipe-998,user-100,NaN,3.297056


In [109]:
sample = sample_all[sample_all['recipe_id'] == 'recipe-1000']
sample = sample.sort_values(by='user_id' ,ascending=True)
sample.head(60)

,recipe_id,user_id,rating_x,rating_y
39191,recipe-1000,user-1,NaN,2.893408
47631,recipe-1000,user-10,NaN,3.089261
129848,recipe-1000,user-100,NaN,2.914120
29974,recipe-1000,user-11,5.0,3.543405
29975,recipe-1000,user-12,2.0,2.445447
50231,recipe-1000,user-13,NaN,3.011924
51239,recipe-1000,user-14,NaN,2.764756
52116,recipe-1000,user-15,NaN,2.991354
53082,recipe-1000,user-16,NaN,2.843145
54085,recipe-1000,user-17,NaN,2.596277


In [114]:
sample = sample.fillna(value=0)
sample.tail()

,recipe_id,user_id,rating_x,rating_y
29998,recipe-1000,user-95,2.0,2.298896
29999,recipe-1000,user-96,1.0,2.137488
127080,recipe-1000,user-97,0.0,2.531210
128029,recipe-1000,user-98,0.0,2.454025
128975,recipe-1000,user-99,0.0,2.663696


# 오류지표

In [111]:
realdata_rating = sample['rating_x'].tolist()
resultdata_rating = sample['rating_y'].tolist()

error_rate_absol = mean_absolute_error(realdata_rating, resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating, resultdata_rating)

### MAE
##### 예측값-결과값 유사 평가 척도
MAE 값 오차의 정도에 관한 값으로 0에 가까울수록 유사도 높음

In [112]:
print("Error Rate(Absolute) : ", error_rate_absol)  # MAE

Error Rate(Absolute) :  2.4094963501249342


### MSE
##### 예측 정확성
MSE 값 적을수록 원본과 예측값의 차이 적어짐 -> 예측 정확성 높음

In [113]:
print("Error Rate(Squared) : ", error_rate_squared)  # MSE
# print("RMSE : ", math.sqrt(error_rate_squared))  # RMSE

Error Rate(Squared) :  6.520571607418482


# correlation

In [115]:
result_list = []
for i in range(0, len(users)):
    user_id = users.iloc[i].user_id
#     recipe_id = result_top10.iloc[i].recipe_id
    predict = predict_rating(user_id, 40, distance_correlation)
#     print(predict)
    
    is_users = ratings['user_id'] == user_id
    users_rating = ratings[is_users]
    exist_recipe = users_rating['recipe_id'].tolist()
    
    for recipe in predict:
        result_list.append([user_id, recipe[0], recipe[1]])
    
resultdf = pd.DataFrame(result_list, columns=['user_id','recipe_id','rating'])
resultdf

,user_id,recipe_id,rating
0,user-1,recipe-1,2.993219
1,user-1,recipe-10,4.130354
2,user-1,recipe-100,2.797510
3,user-1,recipe-1000,2.718601
4,user-1,recipe-1001,2.451811
...,...,...,...
130795,user-100,recipe-995,2.364926
130796,user-100,recipe-996,3.420192
130797,user-100,recipe-997,2.962826
130798,user-100,recipe-998,2.884244


In [116]:
sample_all = pd.merge(left = ratings , right = resultdf, how = "outer", on = ["user_id", 'recipe_id'])
sample_all

,recipe_id,user_id,rating_x,rating_y
0,recipe-1,user-1,3.0,2.993219
1,recipe-1,user-10,4.0,2.450573
2,recipe-1,user-12,1.0,2.018741
3,recipe-1,user-15,1.0,2.837311
4,recipe-1,user-19,4.0,2.981713
...,...,...,...,...
130795,recipe-994,user-100,NaN,2.888611
130796,recipe-995,user-100,NaN,2.364926
130797,recipe-997,user-100,NaN,2.962826
130798,recipe-998,user-100,NaN,2.884244


In [124]:
sample = sample_all[sample_all['recipe_id'] == 'recipe-1000']
sample = sample.sort_values(by='user_id' ,ascending=True)
sample.head(60)

,recipe_id,user_id,rating_x,rating_y
39191,recipe-1000,user-1,NaN,2.718601
47631,recipe-1000,user-10,NaN,3.042101
129848,recipe-1000,user-100,NaN,3.161475
29974,recipe-1000,user-11,5.0,4.077755
29975,recipe-1000,user-12,2.0,2.423670
50231,recipe-1000,user-13,NaN,3.508486
51239,recipe-1000,user-14,NaN,2.987460
52116,recipe-1000,user-15,NaN,2.390444
53082,recipe-1000,user-16,NaN,2.963790
54085,recipe-1000,user-17,NaN,1.875329


In [123]:
sample = sample.fillna(value=0)
sample.tail()

,recipe_id,user_id,rating_x,rating_y
29998,recipe-1000,user-95,2.0,2.736539
29999,recipe-1000,user-96,1.0,2.794618
127080,recipe-1000,user-97,0.0,2.544366
128029,recipe-1000,user-98,0.0,2.509748
128975,recipe-1000,user-99,0.0,2.592496


In [130]:
realdata_rating = sample['rating_x'].tolist()
resultdata_rating = sample['rating_y'].tolist()

error_rate_absol = mean_absolute_error(realdata_rating, resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating, resultdata_rating)

print("Error Rate(Absolute) : ", error_rate_absol)  # MAE
print("Error Rate(Squared) : ", error_rate_squared)  # MSE

Error Rate(Absolute) :  2.3942315148147273
Error Rate(Squared) :  6.321639812186117


# euclidean

In [125]:
result_list = []
for i in range(0, len(users)):
    user_id = users.iloc[i].user_id
#     recipe_id = result_top10.iloc[i].recipe_id
    predict = predict_rating(user_id, 40, distance_euclidean)
#     print(predict)
    
    is_users = ratings['user_id'] == user_id
    users_rating = ratings[is_users]
    exist_recipe = users_rating['recipe_id'].tolist()
    
    for recipe in predict:
        result_list.append([user_id, recipe[0], recipe[1]])
    
resultdf = pd.DataFrame(result_list, columns=['user_id','recipe_id','rating'])
resultdf

,user_id,recipe_id,rating
0,user-1,recipe-1,2.701995
1,user-1,recipe-10,3.586300
2,user-1,recipe-100,2.791312
3,user-1,recipe-1000,2.017556
4,user-1,recipe-1001,2.769953
...,...,...,...
130795,user-100,recipe-995,2.277838
130796,user-100,recipe-996,3.150015
130797,user-100,recipe-997,2.932826
130798,user-100,recipe-998,2.193643


In [127]:
sample_all = pd.merge(left = ratings , right = resultdf, how = "outer", on = ["user_id", 'recipe_id'])
sample = sample_all[sample_all['recipe_id'] == 'recipe-1000']
sample = sample.sort_values(by='user_id' ,ascending=True)
sample.head(60)

,recipe_id,user_id,rating_x,rating_y
39191,recipe-1000,user-1,NaN,2.017556
47631,recipe-1000,user-10,NaN,2.713845
129848,recipe-1000,user-100,NaN,3.000673
29974,recipe-1000,user-11,5.0,2.886112
29975,recipe-1000,user-12,2.0,2.753896
50231,recipe-1000,user-13,NaN,2.951565
51239,recipe-1000,user-14,NaN,2.705317
52116,recipe-1000,user-15,NaN,2.688303
53082,recipe-1000,user-16,NaN,2.789570
54085,recipe-1000,user-17,NaN,2.999183


In [128]:
sample = sample.fillna(value=0)
sample.tail()

,recipe_id,user_id,rating_x,rating_y
29998,recipe-1000,user-95,2.0,2.599913
29999,recipe-1000,user-96,1.0,3.374224
127080,recipe-1000,user-97,0.0,2.698365
128029,recipe-1000,user-98,0.0,2.619498
128975,recipe-1000,user-99,0.0,2.500201


In [129]:
realdata_rating = sample['rating_x'].tolist()
resultdata_rating = sample['rating_y'].tolist()

error_rate_absol = mean_absolute_error(realdata_rating, resultdata_rating)
error_rate_squared = mean_squared_error(realdata_rating, resultdata_rating)

print("Error Rate(Absolute) : ", error_rate_absol)  # MAE
print("Error Rate(Squared) : ", error_rate_squared)  # MSE

Error Rate(Absolute) :  2.3942315148147273
Error Rate(Squared) :  6.321639812186117
